<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Network-in-Network-(NiN)-BLOCK" data-toc-modified-id="Network-in-Network-(NiN)-BLOCK-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Network in Network (NiN) BLOCK</a></span></li></ul></div>

In [1]:
import mxnet
from mxnet import gluon,npx,np,autograd
from mxnet.gluon import nn
npx.set_np()

##  Network in Network (NiN) BLOCK
<img src='../images/nin.jpg'/>
The NiN block consists of one convolutional layer followed by two 1 × 1 convolutional layers that
act as per-pixel fully-connected layers with ReLU activations. The convolution width of the first
layer is typically set by the user. The subsequent widths are fixed to 1 × 1

In [2]:
def nin_block(num_channels, kernel_size, strides, padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size, strides, padding,activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
    return blk

In [3]:
nin = nn.Sequential()
nin.add(nin_block(96, kernel_size=11, strides=4, padding=0),nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256, kernel_size=5, strides=1, padding=2),nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(384, kernel_size=3, strides=1, padding=1),nn.MaxPool2D(pool_size=3, strides=2),
        nn.Dropout(0.5),
        # There are 10 label classes
        nin_block(10, kernel_size=3, strides=1, padding=1),
         # The global average pooling layer automatically sets the window shape
          # to the height and width of the input
           nn.GlobalAvgPool2D(),
           # Transform the four-dimensional output into two-dimensional output
           # with a shape of (batch size, 10)
        nn.Flatten())

In [4]:
X = np.random.uniform(size=(1, 1, 224, 224))
nin.initialize()
for layer in nin:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential1 output shape:	 (1, 96, 54, 54)
pool0 output shape:	 (1, 96, 26, 26)
sequential2 output shape:	 (1, 256, 26, 26)
pool1 output shape:	 (1, 256, 12, 12)
sequential3 output shape:	 (1, 384, 12, 12)
pool2 output shape:	 (1, 384, 5, 5)
dropout0 output shape:	 (1, 384, 5, 5)
sequential4 output shape:	 (1, 10, 5, 5)
pool3 output shape:	 (1, 10, 1, 1)
flatten0 output shape:	 (1, 10)
